# Magic Personality Matcher - Preprocesamiento de datos

Para evitar tener que preprocesar los datos cada vez que abrimos el Google Colab vamos a realizar tanto particionado de los datos como su preprocesamiento en este notebook aparte.

## Fase 1 - Cargar el dataset

Lo primero es cargar el dataset con Pandas y seleccionar la cantidad de filas que queremos usar:

In [1]:
import pandas as pd

raw_data = pd.read_csv("dataset_definitivo.csv")
print("general counts")
print(raw_data['personality'].value_counts())
training_data = raw_data.head(5000)

print("used dataset info")
print(training_data.shape)
print(training_data.head())
print(training_data['personality'].value_counts())

general counts
personality
INTP    27076
INTJ    24299
INFJ    17490
INFP    15248
ENTP    12987
ENFP     7571
ISTP     4088
ENTJ     3465
ENFJ     2242
ESTP     2175
ISTJ     1707
ISFP     1513
ISFJ     1180
ESTJ      602
ESFP      582
ESFJ      328
Name: count, dtype: int64
used dataset info
(5000, 2)
  personality                                               post
0        INTJ  @Pericles216 @HierBeforeTheAC @Sachinettiyil T...
1        INTJ  @Hispanthicckk Being you makes you look cute||...
2        INTJ  @Alshymi Les balles sont réelles et sont tirée...
3        INTJ  I'm like entp but idiotic|||Hey boy, do you wa...
4        INTJ  @kaeshurr1 Give it to @ZargarShanif ... He has...
personality
INFP    811
INFJ    628
INTP    556
INTJ    456
ENFP    429
ENTP    370
ENFJ    330
ISFJ    259
ISFP    229
ISTP    224
ISTJ    204
ENTJ    185
ESFP    119
ESFJ     73
ESTP     67
ESTJ     60
Name: count, dtype: int64


## Fase 2 - Filtrado de Stopwords, Tokenización y Stemming

El preprocesamiento de texto se realiza usando las stopwords y el tokenizer de la librería de lenguaje natural de Python. Primero definimos las stopwords, que serán las de la librería de Python y las personalidades Myers-Briggs para evitar introducir sesgos, como que, si se menciona una personalidad, que esta no se tenga en cuenta para decidir la del autor. Luego se usa el PorterStemmer para obtener la “raíz” de las palabras. Entonces el resultado es una nueva columna que contiene los posts preprocesados.

In [2]:
import multiprocessing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm.contrib.concurrent import process_map
import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()


STOPS = set(stopwords.words("english"))
CUSTOM_STOPS=set(['istj', 'isfj', 'infj', 'intj', 'istp', 'isfp', 'infp', 'intp', 'estp', 'esfp', 'enfp', 'entp', 'estj', 'esfj', 'enfj', 'entj'])

def process_text(post):
    post = post.lower()
    post = re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',post)
    post = re.sub('[^0-9a-z]',' ',post)
    return " ".join([ps.stem(w) for w in word_tokenize(post) if not w in STOPS and w not in CUSTOM_STOPS])

preprocessedData = training_data.loc[:]

num_processes = multiprocessing.cpu_count()

try:
    with multiprocessing.Pool(processes=num_processes) as pool:
        preprocessedData['processed_text'] = process_map(process_text, training_data['post'], max_workers=num_processes, chunksize=10)
finally:
    pool.close()
    pool.join()

preprocessedData

[nltk_data] Downloading package punkt to /home/mario/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  0%|          | 0/5000 [00:00<?, ?it/s]

,personality,post,processed_text
0,INTJ,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,pericles216 hierbeforetheac sachinettiyil pope...
1,INTJ,@Hispanthicckk Being you makes you look cute||...,hispanthicckk make look cute thiccwhiteduk fun...
2,INTJ,@Alshymi Les balles sont réelles et sont tirée...,alshymi le ball sont r ell et sont tir es tr r...
3,INTJ,"I'm like entp but idiotic|||Hey boy, do you wa...",like idiot hey boy want watch twitch kin simon...
4,INTJ,@kaeshurr1 Give it to @ZargarShanif ... He has...,kaeshurr1 give zargarshanif pica sinc childhoo...
...,...,...,...
4995,INTJ,@TheChoirLoft https://t.co/LJjH1v5sKV|||@allie...,thechoirloft got depress feel like lot peopl w...
4996,INTJ,@danrannosaurus Got it. Thank youuu!! Haha|||@...,danrannosauru got thank youuu haha danrannosau...
4997,INTP,WHY☹️☹️☹️☹️ https://t.co/DwJRFJyz6Z|||WHY IS H...,cryinf soop friendcat us disney lucki 4 got ah...
4998,INTP,@sashywaybrights I LOVE THAT SONG OMG|||@ricep...,sashywaybright love song omg ricepoptart marci...


## Fase 3 - Particionar el dataset para entrenamiento y validación
Se divide el dataset en 2 para tener 80% datos de entrenamiento y 20% de prueba

In [3]:
from sklearn.model_selection import train_test_split

X = preprocessedData['processed_text']
Y = preprocessedData['personality']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## Fase 4 - Bolsa de palabras

Se crea una bolsa de palabras utilizando TfidVectorizer, que se basa en la frecuencia de las palabras para determinar su importancia.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Datos de entrenamiento
bagOfWordsModel = TfidfVectorizer()
train_textsBoW = bagOfWordsModel.fit_transform(X_train)
print("X_train bag of words:")
print(train_textsBoW.shape)

# Datos pruebas
test_textsBoW = bagOfWordsModel.transform(X_test)
print("X_test bag of words:")
print(test_textsBoW.shape)

X_train bag of words:
(4000, 207345)
X_test bag of words:
(1000, 207345)


## Fase 5 - Guardar el resultado

Utilizamos joblib para guardar estas bolsas de palabras para poder usarlas para entrenamiento sin tener que repetir el proceso.

In [8]:
from joblib import dump
import numpy as np

dump(train_textsBoW, "train_textsBoW.lzma")
dump(test_textsBoW, "test_textsBoW.lzma")
dump(y_train, "y_train.lzma")
dump(y_test, "y_test.lzma")
dump(bagOfWordsModel, "bag_of_words.lzma")

['bag_of_words.lzma']